In [1]:
import pandas as pd
import numpy as np

### Create Matrix Z

In [ ]:
# Create a matrix Z that aggregate both time and unit fixed effects, and covariates

# Collect unit and time fixed effects in matrix A
time_dummies = pd.get_dummies(df['time'])
unit_dummies = pd.get_dummies(df['unit'])
A = pd.concat([time_dummies, unit_dummies], axis=1)

# Collect covariates in matrix X
X = df[['x1', 'x2']]

# Create matrix Z, the sum of A and X
Z = pd.concat([A, X], axis=1)

# Create matrix Z0, by filtering matrix Z to only include treatment == 0
Z0 = Z[df['treatment'] == 0]

# Create matrix Z1, by filtering matrix Z to only include treatment == 1
Z1 = Z[df['treatment'] == 1]

# Create the matrix multiplication Z1(Z'Z)^-1 Z1'
Z1Z1 = np.matmul(np.matmul(Z1, np.linalg.inv(np.matmul(Z.T, Z))), Z1.T)

# Create the matrix multiplication Z0(Z'Z)^-1 Z1'
Z0Z1 = np.matmul(np.matmul(Z0, np.linalg.inv(np.matmul(Z.T, Z))), Z1.T)

# Create the identity matrix with n= to number of rows with treatment == 1
I = np.identity(Z1.shape[0])

# Create the weights w1 that is equal to the weights we used in the estimand of interest but filtered to only include treatment == 1
w1 = w[df['treatment'] == 1]

# Create the weight v* equal to (I -Z1Z1, Z0Z1) I(I'(I-Z1Z1)I)^-1 I' w1
v_star = np.matmul(np.matmul(I - Z1Z1, Z0Z1), np.linalg.inv(np.matmul(np.matmul(I.T, I - Z1Z1), I)))

### Create the choice \theta_g for error construction

In [ ]:
# Create the treatment effects to retrieve the error term: tau_g = (Sum_i (Sum_t v_star_it) (Sum_t v_star_it tau_it))/ (Sum_i (Sum_t v_star_it)^2)
# In each summation over t, it belongs to G_g, the group of units that are in the same cluster as unit i
tau_g = np.matmul(np.matmul(v_star, df['y']), np.matmul(v_star, df['treatment'])) / np.matmul(np.matmul(v_star, df['treatment']), np.matmul(v_star, df['treatment']))

### Compute the error term

In [ ]:
# Create the error term, which is equal to Y_it - A_i * lambda_i^start - X_it*delta^star - D_it * tau_g
df['error'] = df['y'] - df['unit'].map(lambda x: lambda_start[x]) - df['x1'] * delta_star[0] - df['x2'] * delta_star[1] - df['treatment'] * tau_g

### Compute the variance